# Visualize all senescence scores


## Create one adata with all senescence scores appended
#### Can downsample cells in prol to match tam for visualization and statistics calculations

In [ ]:
adata = sc.read_h5ad('/ix/djishnu/Akanksha/datasets/senescence_fibroblasts/mouse_fibro/MEFs_processed.h5ad')

In [ ]:
adata

In [ ]:
# read obs of adata_scored
obs_scored = pd.read_csv('/ix/djishnu/Akanksha/datasets/senescence_fibroblasts/mouse_fibro/MEFs_DeepScence_scores.txt', sep = '\t')
display(obs_scored)


In [ ]:
# add the ds and binary columns to adata.obs
obs_scored = obs_scored.set_index('Unnamed: 0')  # this is the column with cell names
adata.obs['ds'] = obs_scored['ds']
adata.obs['binary'] = obs_scored['binary']


In [ ]:
adata_tamxoifen = adata[adata.obs['Treatment'] == 'Tam']
adata_control = adata[adata.obs['Treatment'] == 'Prol']


In [ ]:
adata_tamxoifen

In [ ]:
adata_control

In [ ]:
sc.pl.umap(adata, color = ['Treatment','integrated_snn_res.0.4','ds', 'binary'], color_map = 'viridis')

In [ ]:

sc.pl.umap(adata_tamxoifen, color = ['integrated_snn_res.0.4','ds', 'binary'], color_map = 'viridis', title = 'Tamoxifen treated')
sc.pl.umap(adata_control, color = ['integrated_snn_res.0.4','ds', 'binary'], color_map = 'viridis', title = 'Proliferating control')


#### Violin plot of deepscence scores per cell cluster

In [ ]:
# violin plot of ds scores per cell cluster calculated seperately for treated and control cells
sc.pl.violin(adata, keys = ['ds'], groupby = 'Treatment', rotation = 90)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from statsmodels.stats.multitest import multipletests

def add_stat_annotation(x1, x2, y, h, p_adj):
    """Add significance annotation to plot"""
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    if p_adj < 0.001:
        stars = '***'
    elif p_adj < 0.01:
        stars = '**'
    elif p_adj < 0.05:
        stars = '*'
    else:
        stars = 'ns'
    plt.text((x1+x2)*.5, y+h, stars, ha='center', va='bottom')

In [ ]:
# Get data for tamoxifen and proliferating cells
tam_data = adata_tamxoifen.obs[['ds', 'integrated_snn_res.0.4']].assign(Treatment='Tam')
prol_data = adata_control.obs[['ds', 'integrated_snn_res.0.4']].assign(Treatment='Prol')

# Combine the data
plot_data = pd.concat([tam_data, prol_data])

In [ ]:
# Create the plot
plt.figure(figsize=(15, 8))
plt.subplots_adjust(top=0.85)

# Create violin plot
sns.violinplot(data=plot_data, 
              x='integrated_snn_res.0.4',  
              y='ds',                   
              hue='Treatment',             
              split=True,
              inner='quartile')

# Prepare for statistical testing
clusters = sorted(plot_data['integrated_snn_res.0.4'].unique())
max_y = plot_data['ds'].max()
min_y = plot_data['ds'].min()
y_range = max_y - min_y
spacing = y_range * 0.1

# Collect all p-values first
p_values = []
cluster_pairs = []

for cluster in clusters:
    cluster_tam = tam_data[tam_data['integrated_snn_res.0.4'] == cluster]['ds']
    cluster_prol = prol_data[prol_data['integrated_snn_res.0.4'] == cluster]['ds']
    
    # Perform t-test
    stat, p_val = stats.ttest_ind(cluster_tam, cluster_prol)
    p_values.append(p_val)
    cluster_pairs.append((cluster_tam, cluster_prol))

# Correct for multiple testing using Benjamini-Hochberg method
rejected, p_adjusted, _, _ = multipletests(p_values, method='fdr_bh')

# Add annotations with adjusted p-values
for idx, (p_adj, cluster) in enumerate(zip(p_adjusted, clusters)):
    y_pos = max_y + spacing * (1.2)
    add_stat_annotation(idx-0.2, idx+0.2, y_pos, spacing*0.5, p_adj)

# Customize the plot
plt.title('DeepScence Scores by Cluster\n(FDR-corrected p-values)', pad=50)
plt.xlabel('Cluster')
plt.ylabel('DeepScence Score')
plt.xticks(rotation=0)

# Adjust y-axis limits
plt.ylim(min_y - (y_range * 0.1), max_y + (spacing * 3))

# Add legend
plt.legend(title='Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Create the plot
plt.figure(figsize=(15, 8))
plt.subplots_adjust(top=0.85)

# Create violin plot - note removed split=True and adjusted x parameter
sns.violinplot(data=plot_data, 
              x='integrated_snn_res.0.4',  
              y='ds',                   
              hue='Treatment',             
              inner='quartile')

# Prepare for statistical testing
clusters = sorted(plot_data['integrated_snn_res.0.4'].unique())
max_y = plot_data['ds'].max()
min_y = plot_data['ds'].min()
y_range = max_y - min_y
spacing = y_range * 0.1

# Collect all p-values first
p_values = []
cluster_pairs = []

for idx, cluster in enumerate(clusters):
    cluster_tam = tam_data[tam_data['integrated_snn_res.0.4'] == cluster]['ds']
    cluster_prol = prol_data[prol_data['integrated_snn_res.0.4'] == cluster]['ds']
    
    # Perform t-test
    stat, p_val = stats.ttest_ind(cluster_tam, cluster_prol)
    p_values.append(p_val)
    cluster_pairs.append((cluster_tam, cluster_prol))
    
    # Adjust x-coordinates for side-by-side violins
    x1 = idx - 0.2  # Position of first violin in pair
    x2 = idx + 0.2  # Position of second violin in pair
    
    # Add statistical annotation
    y_pos = max_y + spacing * (1.2)
    add_stat_annotation(x1, x2, y_pos, spacing*0.5, p_val)

# Correct for multiple testing using Benjamini-Hochberg method
rejected, p_adjusted, _, _ = multipletests(p_values, method='fdr_bh')

# Customize the plot
plt.title('DeepScence Scores by Cluster\n(FDR-corrected p-values)', pad=50)
plt.xlabel('Cluster')
plt.ylabel('DeepScence Score')
plt.xticks(rotation=0)

# Adjust y-axis limits
plt.ylim(min_y - (y_range * 0.1), max_y + (spacing * 3))

# Add legend
plt.legend(title='Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate percentage of senescent cells per cluster for tamoxifen treated cells
cluster_percentages_tam = (adata_tamxoifen.obs
    .groupby('integrated_snn_res.0.4')
    .agg({
        'binary': ['count', 'sum']
    })
)
cluster_percentages_tam['percent_senescent'] = (cluster_percentages_tam['binary']['sum'] / 
                                              cluster_percentages_tam['binary']['count'] * 100)

# Calculate percentage of senescent cells per cluster for proliferating control cells
cluster_percentages_prol = (adata_control.obs
    .groupby('integrated_snn_res.0.4')
    .agg({
        'binary': ['count', 'sum']
    })
)
cluster_percentages_prol['percent_senescent'] = (cluster_percentages_prol['binary']['sum'] / 
                                               cluster_percentages_prol['binary']['count'] * 100)

# Display results
print("Tamoxifen treated cells - % senescent cells per cluster:")
print(cluster_percentages_tam['percent_senescent'].round(2))
print("\nProliferating control cells - % senescent cells per cluster:")
print(cluster_percentages_prol['percent_senescent'].round(2))


### Viz sen pathways from Aucell 

In [ ]:
sen_pathways = ["SAUL_SEN_MAYO", "GOBP_REPLICATIVE_SENESCENCE", "REACTOME_DNA_DAMAGE_TELOMERE_STRESS_INDUCED_SENESCENCE", "REACTOME_FORMATION_OF_SENESCENCE_ASSOCIATED_HETEROCHROMATIN_FOCI_SAHF"]
adata.obs[sen_pathways] = adata.obsm["aucell_estimate"][sen_pathways]

In [ ]:
sc.pl.umap(
    adata,
    color=["Treatment", "integrated_snn_res.0.4"] + sen_pathways,
    frameon=False,
    ncols=2,
    wspace=0.3,
)

In [ ]:
sc.pl.umap(
    adata_tamxoifen,
    color=["integrated_snn_res.0.4"] + sen_pathways,
    frameon=False,
    ncols=2,
    wspace=0.3,
    title="Tamoxifen treated"
)
sc.pl.umap(
    adata_control,
    color=["integrated_snn_res.0.4"] + sen_pathways,
    frameon=False,
    ncols=2,
    wspace=0.3,
    title="Proliferating control"
)

#### Heatmaps of geneset scores/methods to get senescence transcriptomic signatures

In [ ]:
# group adata obs by cell cluster (integrated_snn_res.0.4) and get mean value of scores of sen_pathways
sen_tam_scores_mean = adata_tamxoifen.obs.groupby("integrated_snn_res.0.4")[sen_pathways].mean()
sen_prol_scores_mean = adata_control.obs.groupby("integrated_snn_res.0.4")[sen_pathways].mean()
display(sen_tam_scores_mean)
display(sen_prol_scores_mean)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot heatmap for tamoxifen treated cells
plt.figure(figsize=(10, 10))
sns.heatmap(sen_tam_scores_mean.T,
            cmap='YlOrRd',  # Changed to YlOrRd colormap
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Score'},
            xticklabels=True,
            yticklabels=True,
            vmin=0,  # Set minimum value for consistent scale
            vmax=0.2)  # Set maximum value for consistent scale
plt.title('Senescence Scores - Tamoxifen Treated')
plt.xlabel('Cell Clusters')
plt.ylabel('Senescence Pathways')
plt.show()

# Plot heatmap for proliferating control cells
plt.figure(figsize=(10, 10))
sns.heatmap(sen_prol_scores_mean.T,
            cmap='YlOrRd',  # Changed to YlOrRd colormap
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Score'},
            xticklabels=True,
            yticklabels=True,
            vmin=0,  # Set minimum value for consistent scale
            vmax=0.2)  # Set maximum value for consistent scale
plt.title('Senescence Scores - Proliferating Control')
plt.xlabel('Cell Clusters')
plt.ylabel('Senescence Pathways')
plt.show()

#### Violin plots to compare senescence scores between treated and control for geneset scores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from statsmodels.stats.multitest import multipletests

def add_stat_annotation(x1, x2, y, h, p_adj):
    """Add significance annotation to plot"""
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    if p_adj < 0.001:
        stars = '***'
    elif p_adj < 0.01:
        stars = '**'
    elif p_adj < 0.05:
        stars = '*'
    else:
        stars = 'ns'
    plt.text((x1+x2)*.5, y+h, stars, ha='center', va='bottom')

# Plot each pathway separately
for pathway in sen_pathways:
    # Get data for tamoxifen and proliferating cells
    tam_data = adata_tamxoifen.obs[[pathway, 'integrated_snn_res.0.4']].assign(Treatment='Tam')
    prol_data = adata_control.obs[[pathway, 'integrated_snn_res.0.4']].assign(Treatment='Prol')
    
    # Combine the data
    plot_data = pd.concat([tam_data, prol_data])
    
    # Create the plot
    plt.figure(figsize=(15, 8))
    plt.subplots_adjust(top=0.85)
    
    # Create violin plot
    sns.violinplot(data=plot_data, 
                  x='integrated_snn_res.0.4',  
                  y=pathway,                   
                  hue='Treatment',             
                  split=True,
                  inner='quartile')
    
    # Prepare for statistical testing
    clusters = sorted(plot_data['integrated_snn_res.0.4'].unique())
    max_y = plot_data[pathway].max()
    min_y = plot_data[pathway].min()
    y_range = max_y - min_y
    spacing = y_range * 0.1
    
    # Collect all p-values first
    p_values = []
    cluster_pairs = []
    
    for cluster in clusters:
        cluster_tam = tam_data[tam_data['integrated_snn_res.0.4'] == cluster][pathway]
        cluster_prol = prol_data[prol_data['integrated_snn_res.0.4'] == cluster][pathway]
        
        # Perform t-test
        stat, p_val = stats.ttest_ind(cluster_tam, cluster_prol)
        p_values.append(p_val)
        cluster_pairs.append((cluster_tam, cluster_prol))
    
    # Correct for multiple testing using Benjamini-Hochberg method
    rejected, p_adjusted, _, _ = multipletests(p_values, method='fdr_bh')
    
    # Add annotations with adjusted p-values
    for idx, (p_adj, cluster) in enumerate(zip(p_adjusted, clusters)):
        y_pos = max_y + spacing * (1.2)
        add_stat_annotation(idx-0.2, idx+0.2, y_pos, spacing*0.5, p_adj)
    
    # Customize the plot
    plt.title(f'{pathway} Scores by Cluster\n(FDR-corrected p-values)', pad=50)
    plt.xlabel('Cluster')
    plt.ylabel('Score')
    plt.xticks(rotation=0)
    
    # Adjust y-axis limits
    plt.ylim(min_y - (y_range * 0.1), max_y + (spacing * 3))
    
    # Add legend
    plt.legend(title='Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

### SID scores

In [ ]:
# load adata with SID
adata_with_SID = sc.read('/ix/djishnu/Akanksha/datasets/senescence_fibroblasts/tamoxifen_with_SID.h5ad')
adata_with_SID


In [ ]:
# load pred_dict from pickle file
import pickle
with open('/ix/djishnu/Akanksha/datasets/senescence_fibroblasts/tam_SenCID_pred_dict.pkl', 'rb') as f:
    pred_dict = pickle.load(f)
pred_dict


In [ ]:
markers = ['rec_SID1', 'rec_SID2', 'rec_SID3', 'rec_SID4', 'rec_SID5', 'rec_SID6']
sc.pl.violin(adata_with_SID, keys = markers)

In [ ]:
adata.obs['SID3_Score'] = pred_dict['SID3'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID3_Score'], color_map = 'viridis', vmax = 0.5)


In [ ]:
adata.obs['SID5_Score'] = pred_dict['SID5'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID5_Score'], color_map = 'viridis')


In [ ]:
adata.obs['SID1_Score'] = pred_dict['SID1'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID1_Score'], color_map = 'viridis')


In [ ]:
adata.obs['SID2_Score'] = pred_dict['SID2'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID2_Score'], color_map = 'viridis')



In [ ]:
adata.obs['SID4_Score'] = pred_dict['SID4'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID4_Score'], color_map = 'viridis')


In [ ]:
adata.obs['SID6_Score'] = pred_dict['SID6'].loc[adata.obs_names, 'SID_Score']
# color bar should have min and max of the score values 
sc.pl.umap(adata, color = ['SID6_Score'], color_map = 'viridis')


#### Heatmap of gene expression of custom gene list

In [ ]:
# custom geneset from NIA (human fibroblasts)
NIA_sen_genes = [
    "CKS2",
    "CCNB1",
    "PTTG1", 
    "QSOX1",
    "PARP1",
    "LBR",
    "LMNB1",
    "UQCR11",
    "NDUFA1", 
    "NDUFA3",
    "COX6B1",
    "CST3",
    "TGFB1",
    "PALLD",
    "CCND1",
    "IGFBP7",
    "SSRP1",
    "ANP32B",
    "PINK1",
    "BCL2L2"
]
# convert to mouse genes using gtf files of both species and ortholog information file
ortholog_info = pd.read_csv('/ix/djishnu/Akanksha/datasets/gene_sets/human_mouse_1to1_orthologs.csv')
ortholog_info


In [ ]:
# search for the human genes in the ortholog df and return the mouse orthologs
mouse_NIA_sen_genes = ortholog_info[ortholog_info['human'].isin(NIA_sen_genes)]['mouse'].tolist()
display(mouse_NIA_sen_genes)

# display the human genes not in the ortholog df
human_NIA_sen_genes_not_in_ortholog = set(NIA_sen_genes) - set(ortholog_info['human'])
display(human_NIA_sen_genes_not_in_ortholog)

# append curated genes to the mouse_NIA_sen_genes list
ortholog_genes_mouse = ['Anp32b','Palld', 'Pttg1']
mouse_NIA_sen_genes = mouse_NIA_sen_genes + ortholog_genes_mouse
display(mouse_NIA_sen_genes)



In [ ]:
# scanpy heatmap of the mouse NIA senescence genes for both treated and control adata
sc.pl.heatmap(adata_tamxoifen, var_names = mouse_NIA_sen_genes, groupby = 'integrated_snn_res.0.4', figsize = (10,10))
sc.pl.heatmap(adata_control, var_names = mouse_NIA_sen_genes, groupby = 'integrated_snn_res.0.4', figsize = (10,10))


In [ ]:
sc.pl.matrixplot(
    adata_tamxoifen,
    mouse_NIA_sen_genes,
    "integrated_snn_res.0.4",
    dendrogram=True,
    cmap="Blues",
    standard_scale="var",
    colorbar_title="column scaled\nexpression",
    title = "Tamoxifen treated"
    #return_fig = True
)
#mp_tam.add_totals().style(edge_color='black').show()
sc.pl.matrixplot(
    adata_control,
    mouse_NIA_sen_genes,
    "integrated_snn_res.0.4",
    dendrogram=True,
    cmap="Blues",
    standard_scale="var",
    colorbar_title="column scaled\nexpression",
    title = "Proliferating control"
    #return_fig = True
)
#mp_prol.add_totals().style(edge_color='black').show()


In [ ]:
# plot matrix plot for all cells
sc.pl.matrixplot(adata, mouse_NIA_sen_genes, "integrated_snn_res.0.4", dendrogram=True, cmap="Blues", standard_scale="var", colorbar_title="column scaled\nexpression", title = "Integrated Tamoxifen Treated and Proliferating Control")
#mp_all.add_totals().style(edge_color='black').show()